In [1]:
# Update sklearn to prevent version mismatches
!pip install sklearn --upgrade

  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1316 sha256=2befafc76e0523d0391c1397f1ef5facbd9adf8d5a805bd5eada80e188599fed
  Stored in directory: /Users/amosfung/Library/Caches/pip/wheels/76/03/bb/589d421d27431bcd2c6da284d5f2286c8e3b2ea3cf1594c074
Successfully built sklearn


In [1]:
# install joblib. This will be used to save your model. 
# Restart your kernel after installing 
!pip install joblib

In [1]:
# dependencies
import pandas as pd
import numpy as np

# Read the CSV and Perform Basic Data Cleaning

In [2]:
df = pd.read_csv("exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


# Select your features (columns)

In [3]:
# Set features. This will also be used as your x values.
selected_features = df[['koi_period', 'koi_prad', 'koi_fpflag_nt', 'koi_steff', 'koi_slogg', 'koi_srad']]


# Create a Train Test Split

Use `koi_disposition` for the y values

In [15]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(selected_features, df['koi_disposition'])

# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [36]:
# Process data for dnn
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import MinMaxScaler, LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(df['koi_disposition'])
y_train_encoded = label_encoder.transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

y_train_categorical = to_categorical(y_train_encoded)
y_test_categorical = to_categorical(y_train_encoded)


X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [23]:
y_train_categorical.shape

(5243, 3)

# Train the Model



In [40]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier

# function to create deep learning model with sklearn wrapper
# def make_model():
#     deep_neural_model = Sequential()
#     deep_neural_model.add(Dense(100, activation = 'relu', input_dim = 6))
#     deep_neural_model.add(Dense(100, activation = 'relu'))
#     deep_neural_model.add(Dense(3, activation = 'softmax'))
#     deep_neural_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
#     return deep_neural_model

# deep_neural_model = KerasClassifier(build_fn=make_model, epochs = 100, batch_size =10, verbose=2)

deep_neural_model = Sequential()
deep_neural_model.add(Dense(100, activation = 'relu', input_dim = 6))
deep_neural_model.add(Dense(100, activation = 'relu'))
deep_neural_model.add(Dense(3, activation = 'softmax'))
deep_neural_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

deep_neural_model.fit(X_train_scaled, y_train_categorical, epochs=50, shuffle = True, verbose=2)


Epoch 1/50
 - 0s - loss: 0.9434 - acc: 0.5085
Epoch 2/50
 - 0s - loss: 0.8873 - acc: 0.5400
Epoch 3/50
 - 0s - loss: 0.8806 - acc: 0.5487
Epoch 4/50
 - 0s - loss: 0.8776 - acc: 0.5508
Epoch 5/50
 - 0s - loss: 0.8707 - acc: 0.5556
Epoch 6/50
 - 0s - loss: 0.8699 - acc: 0.5566
Epoch 7/50
 - 0s - loss: 0.8686 - acc: 0.5607
Epoch 8/50
 - 0s - loss: 0.8642 - acc: 0.5644
Epoch 9/50
 - 0s - loss: 0.8642 - acc: 0.5592
Epoch 10/50
 - 0s - loss: 0.8625 - acc: 0.5609
Epoch 11/50
 - 0s - loss: 0.8598 - acc: 0.5613
Epoch 12/50
 - 0s - loss: 0.8551 - acc: 0.5678
Epoch 13/50
 - 0s - loss: 0.8550 - acc: 0.5697
Epoch 14/50
 - 0s - loss: 0.8505 - acc: 0.5716
Epoch 15/50
 - 0s - loss: 0.8534 - acc: 0.5678
Epoch 16/50
 - 0s - loss: 0.8476 - acc: 0.5773
Epoch 17/50
 - 0s - loss: 0.8478 - acc: 0.5787
Epoch 18/50
 - 0s - loss: 0.8453 - acc: 0.5762
Epoch 19/50
 - 0s - loss: 0.8435 - acc: 0.5848
Epoch 20/50
 - 0s - loss: 0.8392 - acc: 0.5834
Epoch 21/50
 - 0s - loss: 0.8387 - acc: 0.5857
Epoch 22/50
 - 0s - lo

In [25]:
model_train_loss, model_train_accuracy = deep_neural_model.evaluate(X_train_scaled, y_train_categorical)
model_test_loss, model_test_accuracy = deep_neural_model.evaluate(X_test_scaled, y_test_categorical)

print(f"Training Data Loss: {model_train_loss}, Training Data Accuracy: {model_train_accuracy}")
print(f"Testing Data Loss: {model_test_loss}, Testing Data Accuracy: {model_test_accuracy}")

1748/1748 [==============================] - 0s 23us/sample - loss: 0.7963 - acc: 0.6270
Training Data Loss: 0.7794291235530693, Training Data Accuracy: 0.6273126006126404
Testing Data Loss: 0.7963326492898797, Testing Data Accuracy: 0.6270022988319397


# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [37]:
# Create the GridSearchCV model
from sklearn.model_selection import GridSearchCV
from tensorflow.keras import optimizers
params = {
    'batch_size': [10, 20, 30, 40, 50, 100],
    'epochs': [10, 50, 100]
}

grid = GridSearchCV(estimator=deep_neural_model, param_grid=params, scoring= 'accuracy',verbose = 2, n_jobs=-1, cv=3)
grid.fit(X_train_scaled, y_train_encoded)

Fitting 3 folds for each of 18 candidates, totalling 54 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  3.9min
[Parallel(n_jobs=-1)]: Done  54 out of  54 | elapsed:  4.9min finished
Epoch 1/100
 - 1s - loss: 0.9270 - acc: 0.5199
Epoch 2/100
 - 0s - loss: 0.8848 - acc: 0.5442
Epoch 3/100
 - 0s - loss: 0.8775 - acc: 0.5525
Epoch 4/100
 - 0s - loss: 0.8741 - acc: 0.5598
Epoch 5/100
 - 0s - loss: 0.8693 - acc: 0.5573
Epoch 6/100
 - 0s - loss: 0.8648 - acc: 0.5646
Epoch 7/100
 - 0s - loss: 0.8661 - acc: 0.5598
Epoch 8/100
 - 0s - loss: 0.8594 - acc: 0.5621
Epoch 9/100
 - 0s - loss: 0.8578 - acc: 0.5716
Epoch 10/100
 - 0s - loss: 0.8583 - acc: 0.5712
Epoch 11/100
 - 0s - loss: 0.8550 - acc: 0.5728
Epoch 12/100
 - 0s - loss: 0.8495 - acc: 0.5739
Epoch 13/100
 - 0s - loss: 0.8466 - acc: 0.5739
Epoch 14/100
 - 0s - loss: 0.8450 - acc: 0.5777
Epoch 15/100
 - 0s - loss: 0.8462 - acc: 0.5819
Epoch 16/100
 - 0s 

GridSearchCV(cv=3, error_score='raise-deprecating',
             estimator=<keras.wrappers.scikit_learn.KerasClassifier object at 0x1a335e41d0>,
             iid='warn', n_jobs=-1,
             param_grid={'batch_size': [10, 20, 30, 40, 50, 100],
                         'epochs': [10, 50, 100]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='accuracy', verbose=2)

In [ ]:
# Train the model with GridSearch

In [38]:
print(grid.best_params_)
print(grid.best_score_)

{'batch_size': 20, 'epochs': 100}
0.6469578485599847


# Save the Model

In [41]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash

filename = 'exoplanet_dnn_model.h5'
deep_neural_model.save(filename)